In [ ]:
! pip install pydub

In [ ]:
import numpy as np
import pandas as pd
from pydub import AudioSegment
from pydub.generators import Sine, Square, Sawtooth
import torch

# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

# Function to calculate brightness from RGB
def brightness(r, g, b):
    return 0.299 * r + 0.587 * g + 0.114 * b

# Function to map brightness to pitch
def brightness_to_pitch(brightness):
    # Map brightness (0-255) to frequency (220-880 Hz)
    return 220 + (brightness / 255.0) * 660

# Function to generate sound from RGB
def rgb_to_sound(r, g, b, duration=100):
    pitch = brightness_to_pitch(brightness(r, g, b))

    # Choose sound based on dominant color
    if r > g and r > b:
        tone = Sine(pitch).to_audio_segment(duration=duration)
    elif g > r and g > b:
        tone = Square(pitch).to_audio_segment(duration=duration)
    else:
        tone = Sawtooth(pitch).to_audio_segment(duration=duration)

    return tone

# Load RGB values from CSV
df = pd.read_csv('/content/galaxy_pixel_data.csv')  # Replace with your CSV file path

# Take the first 1000 rows
df_subset = df.head(1100)

# Convert to PyTorch tensors and move to GPU
r_tensor = torch.tensor(df_subset['R'].values, dtype=torch.float32).to(device)
g_tensor = torch.tensor(df_subset['G'].values, dtype=torch.float32).to(device)
b_tensor = torch.tensor(df_subset['B'].values, dtype=torch.float32).to(device)

# Create an empty audio segment
audio = AudioSegment.silent(duration=0)

# Generate sound for each pixel in the subset
for r, g, b in zip(r_tensor, g_tensor, b_tensor):
    tone = rgb_to_sound(r.item(), g.item(), b.item())
    audio += tone

# Export the final sound file
output_filename = "output_sound_1100_rows_gpu.wav"
audio.export(output_filename, format="wav")
print(f"Generated {output_filename}")



Using device: cuda
Generated output_sound_1000_rows_gpu.wav


In [ ]:
import numpy as np
import pandas as pd
from pydub import AudioSegment
from pydub.generators import Sine

# Function to calculate brightness from RGB
def brightness(r, g, b):
    return 0.299 * r + 0.587 * g + 0.114 * b

# Function to map brightness to frequency
def brightness_to_frequency(brightness):
    # Map brightness (0-255) to frequency (220-880 Hz)
    return 220 + (brightness / 255.0) * 660

# Function to map RGB sum to amplitude
def rgb_to_amplitude(r, g, b):
    # Map RGB sum (0-765) to amplitude (0-1)
    return (r + g + b) / 765.0

# Function to map x-coordinate to stereo panning
def x_to_pan(x, max_x):
    # Map x-coordinate (0-max_x) to panning (-1.0 to 1.0)
    return (x / max_x) * 2 - 1

# Function to generate sound from RGB and x-coordinate
def pixel_to_sound(r, g, b, x, max_x, duration=100):
    brightness_value = brightness(r, g, b)
    frequency = brightness_to_frequency(brightness_value)
    amplitude = rgb_to_amplitude(r, g, b)
    pan = x_to_pan(x, max_x)

    # Generate sine wave with the calculated frequency and amplitude
    tone = Sine(frequency).to_audio_segment(duration=duration).apply_gain(amplitude * 20 - 20)  # Apply gain to match amplitude

    # Pan the sound
    tone = tone.pan(pan)

    return tone

# Load RGB values from CSV
df = pd.read_csv('/content/galaxy_pixel_data.csv')  # Replace with your CSV file path

# Take the first 1000 rows and add an x-coordinate column
df_subset = df.head(1100)
df_subset['X'] = np.linspace(0, len(df_subset) - 1, len(df_subset))

# Create an empty audio segment
audio = AudioSegment.silent(duration=0)

# Generate sound for each pixel in the subset
max_x = len(df_subset) - 1
for index, row in df_subset.iterrows():
    r, g, b, x = row['R'], row['G'], row['B'], row['X']
    tone = pixel_to_sound(r, g, b, x, max_x)
    audio += tone

# Export the final sound file
output_filename = "output2_sound_1100_rows_mapping.wav"
audio.export(output_filename, format="wav")
print(f"Generated {output_filename}")



<ipython-input-4-61909545e456>:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['X'] = np.linspace(0, len(df_subset) - 1, len(df_subset))


Generated output2_sound_1100_rows_mapping.wav


In [ ]:
import numpy as np
import pandas as pd
from pydub import AudioSegment
from pydub.generators import Sine
import colorsys

# Function to convert RGB to HSV
def rgb_to_hsv(r, g, b):
    return colorsys.rgb_to_hsv(r / 255.0, g / 255.0, b / 255.0)

# Function to map hue to frequency
def hue_to_frequency(hue):
    # Map hue (0-1) to frequency (220-880 Hz)
    return 220 + hue * 660

# Function to map saturation to amplitude
def saturation_to_amplitude(saturation):
    # Map saturation (0-1) to amplitude (0-1)
    return saturation

# Function to map value to duration
def value_to_duration(value):
    # Map value (0-1) to duration (100-500 ms)
    return 100 + value * 400

# Function to generate sound from HSV
def hsv_to_sound(hue, saturation, value):
    frequency = hue_to_frequency(hue)
    amplitude = saturation_to_amplitude(saturation)
    duration = value_to_duration(value)

    # Generate sine wave with the calculated frequency and amplitude
    tone = Sine(frequency).to_audio_segment(duration=duration).apply_gain(amplitude * 20 - 20)  # Apply gain to match amplitude

    return tone

# Load RGB values from CSV
df = pd.read_csv('/content/galaxy_pixel_data.csv')  # Replace with your CSV file path

# Take the first 1000 rows
df_subset = df.head(1100)

# Create an empty audio segment
audio = AudioSegment.silent(duration=0)

# Generate sound for each pixel in the subset
for index, row in df_subset.iterrows():
    r, g, b = row['R'], row['G'], row['B']
    hue, saturation, value = rgb_to_hsv(r, g, b)
    tone = hsv_to_sound(hue, saturation, value)
    audio += tone

# Export the final sound file
output_filename = "output3_sound_1100_rows_hsv_mapping.wav"
audio.export(output_filename, format="wav")
print(f"Generated {output_filename}")



Generated output3_sound_1100_rows_hsv_mapping.wav
